# Quickstart

## Installation

In [ ]:
%pip install "fastrepl==0.0.8"

You can find all releases [here](https://pypi.org/project/fastrepl).

## Setup FastREPL

In [1]:
# When using it in a script
import fastrepl

# When using it in a notebook
import fastrepl.repl as fastrepl

## Prepare Dataset

We will use [daily_dialog](https://huggingface.co/datasets/daily_dialog) from Huggingface.

In [2]:
import re
from datasets import load_dataset

ds = load_dataset("daily_dialog", split="test")
ds = ds.shuffle(4)
ds = ds.select(range(50))


def clean(text):
    return re.sub(r"\s+([,.'!?])", r"\1", text.strip())


def get_input(row):
    msgs = [clean(msg) for msg in row["dialog"]]
    row["sample"] = "\n".join(msgs)  # `SimpleEvaluator` expect `sample` column

    return row


ds = ds.map(get_input, remove_columns=["dialog", "act", "emotion"])
ds

Dataset({
    features: ['sample'],
    num_rows: 50
})

## Define Evaluator

Here, we are doing simple classifiction, but there are two interesting points.

1. You can pass nearly any model for evaluation. (Thanks to [LiteLLM](https://github.com/BerriAI/litellm)).
2. **`fastrepl` enhances accuracy by reducing [bias](/guides/dealing_with_bias.md)**. `position_debias_strategy` is one example, which ensures that the order of labels doesn't affect the outcome.

In [3]:
evaluator = fastrepl.SimpleEvaluator(
    node=fastrepl.LLMClassificationHead(
        model="gpt-3.5-turbo",
        context="You will receive casual conversation between two people.",
        labels={
            "FUN": "at least one of the two people try to be funny and entertain.",
            "NOT_FUN": "given conversation lacks humor or entertainment value.",
        },
        position_debias_strategy="consensus",
    ),
)

## Run Evaluator

Here are some notes about running the evaluator:

1. `ThreadPool` is used to make it faster (controlled by the `NUM_THREADS` [environment variable](/getting_started/env.md)).
2. Any errors from different LLM providers are properly handled and retried with backoff if necessary.
3. Since we passed `num=2` to `run()`, it will execute same evaluation twice, and return two results.

In [5]:
result = fastrepl.LocalRunner(evaluator=evaluator, dataset=ds).run(num=2)
result

Output()

Dataset({
    features: ['sample', 'result'],
    num_rows: 50
})

In [6]:
result.to_pandas()[:10]

,sample,result
0,"Would you like to take a look at the menu, sir...","[NOT_FUN, NOT_FUN]"
1,Help! Help!\nWhat's the matter?,"[NOT_FUN, None]"
2,"Whatever we do, we should do it above board.\n...","[None, None]"
3,"May I see your passport, please?\nCertainly. H...","[NOT_FUN, NOT_FUN]"
4,We're thinking about going to America.\nHave y...,"[NOT_FUN, NOT_FUN]"
5,"Do you believe in UFOs?\nOf course, they are o...","[FUN, FUN]"
6,What do you think about the equipment in our c...,"[NOT_FUN, NOT_FUN]"
7,How was your business trip?\nGreat - they wine...,"[FUN, FUN]"
8,"Hello, Parker. How ’ s everything?\nCan ’ t co...","[None, None]"
9,Our toner cartridges are already out of ink......,"[NOT_FUN, NOT_FUN]"


One interesting point to note is that, due to the `position_debias_strategy="consensus"`, if the order of the labels affects the result, `fastrepl` will return `None`. We'll be returning more meaningful value in the later version of `fastrepl`.

In [8]:
print([row[0] for row in result["result"]].count(None))
print([row[1] for row in result["result"]].count(None))

4
7


Now we got some numbers.

In [9]:
def metric(result):
    f = result.count("FUN")
    nf = result.count("NOT_FUN")
    return f / (f + nf)


print(metric([row[0] for row in result["result"]]))
print(metric([row[1] for row in result["result"]]))

0.30434782608695654
0.27906976744186046


`fastrepl` has built-in support for measuring `inter-rater reliability` when `num > 1`. If `num = 2`, it use `Cohen's Kappa`, otherwise, it use `Fleiss' Kappa`. 

In [10]:
fastrepl.Analyzer(result).run(mode="kappa")

{'kappa': -0.017678756638306883}

Although the `metric` we obtained is quite similar, the `Kappa value` indicates that there is poor agreement between the two runs.

We will revisit this in the future.